In [1]:
!pip install "transformers >= 4.26.0"

In [2]:
import matplotlib.pyplot as plt
import json
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [3]:
questions = json.load(open(f'/kaggle/input/vqa-v2-val/VQA_v2_Val/v2_OpenEnded_mscoco_val2014_questions.json'))['questions'][107177:]
answers = json.load(open(f'/kaggle/input/vqa-v2-val/VQA_v2_Val/v2_mscoco_val2014_annotations.json'))['annotations'][107177:]

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
def load_image(image_dir):
    raw_image = Image.open(image_dir).convert('RGB')   
    return raw_image

In [6]:
id2image = {}
for question in tqdm(questions):
    image_id = str(question['image_id'])
    image_dir = f"/kaggle/input/vqa-v2-val/VQA_v2_Val/val2014/val2014/COCO_val2014_{(12-len(image_id)) * '0' + image_id}.jpg"
    if image_id not in id2image.keys():
        image = load_image(image_dir)
        id2image[image_id] = load_image(image_dir)

100%|██████████| 107177/107177 [04:40<00:00, 382.51it/s]


In [7]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

2024-05-12 18:00:12.217887: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 18:00:12.218024: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 18:00:12.310739: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [8]:
prediction = []

In [ ]:
for q in tqdm(questions):
    image_id = str(q['image_id'])
    question = q['question']
    question_id = q['question_id']
    raw_image = id2image[image_id]
    inputs = processor(raw_image, question, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    answer = processor.decode(out[0], skip_special_tokens=True)
    prediction.append({
        "question_id": question_id,
        "answer": answer
    })

 17%|█▋        | 18738/107177 [31:22<2:25:00, 10.16it/s]

In [ ]:
json.dump(prediction, open(f'/kaggle/working/Fold_{2}_Pred.json', 'w'))